In [1]:
import os
os.remove('snorkel.db')

In [2]:
from snorkel import SnorkelSession
session = SnorkelSession()

import os, sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

In [3]:
import os
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser

doc_parser = HTMLParser(path=os.environ['SNORKELHOME'] + '/test/data/table_test/diseases.xhtml')
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser, max_docs=1)

In [4]:
%time corpus = cp.parse_corpus(name='Sandbox', session=session)
print "%d Phrases in %s" % (
    len([phrase for doc in corpus.documents for phrase in doc.phrases]), corpus)

[========================================] 100%

CPU times: user 293 ms, sys: 26.2 ms, total: 319 ms
Wall time: 751 ms
33 Phrases in Corpus (Sandbox)


In [5]:
from snorkel.models import candidate_subclass

# Year = candidate_subclass('Year', ['year'])
Disease = candidate_subclass('Disease', ['disease'])
# Disease_Year = candidate_subclass('Disease_Year', ['disease','year'])

In [6]:
from snorkel.candidates import OmniNgrams
from hardware_utils import OmniNgramsHardware

omni_ngrams = OmniNgrams(n_max=3, split_tokens=None)
omni_hardware = OmniNgramsHardware(n_max=3, split_tokens=None)

In [7]:
diseases = ['viruses', 'coughs', 'colds', 'brain', 'cancer', 
              'brain cancer', 'disease', 'location', 'polio', 
              'chicken pox', 'plague', 'scurvy', 'infectious diseases', 
              'arthritis', 'yellow fever', 'fever', 'hypochondria', 'pneumonia']
print "Loaded %d diseases." % len(diseases)

Loaded 18 diseases.


In [8]:
from snorkel.matchers import DictionaryMatch, RegexMatchEach, DateMatcher

disease_matcher = DictionaryMatch(d=diseases, ignore_case=True)
year_matcher = DateMatcher()
part_matcher = RegexMatchEach(rgx='BC.*')

In [9]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Disease, [omni_ngrams], [disease_matcher])
# ce = CandidateExtractor(Disease_Year, 
#                         [omni_ngrams, omni_hardware], 
#                         [disease_matcher, part_matcher])

In [10]:
%time candidates = ce.extract(corpus.documents, 'Sandbox Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%

CPU times: user 103 ms, sys: 7.29 ms, total: 110 ms
Wall time: 163 ms
Candidate Set (Sandbox Candidates) contains 16 Candidates


In [11]:
for c in candidates: 
    print c

Disease(Span("Location", parent=35, chars=[0,7], words=[0,0]))
Disease(Span("viruses", parent=2, chars=[9,15], words=[2,2]))
Disease(Span("Disease", parent=34, chars=[0,6], words=[0,0]))
Disease(Span("Brain", parent=3, chars=[0,4], words=[0,0]))
Disease(Span("plague", parent=43, chars=[4,9], words=[1,1]))
Disease(Span("colds", parent=2, chars=[30,34], words=[7,7]))
Disease(Span("Infectious diseases", parent=20, chars=[9,27], words=[3,4]))
Disease(Span("Brain Cancer", parent=3, chars=[0,11], words=[0,1]))
Disease(Span("Scurvy", parent=45, chars=[0,5], words=[0,0]))
Disease(Span("Cancer", parent=3, chars=[6,11], words=[1,1]))
Disease(Span("Polio", parent=37, chars=[0,4], words=[0,0]))
Disease(Span("Hypochondria", parent=58, chars=[0,11], words=[0,0]))
Disease(Span("Fever", parent=55, chars=[0,4], words=[0,0]))
Disease(Span("Pneumonia", parent=33, chars=[27,35], words=[7,7]))
Disease(Span("Arthritis", parent=51, chars=[0,8], words=[0,0]))
Disease(Span("coughs", parent=2, chars=[18,23], wo

In [12]:
# from snorkel.annotations import FeatureManager

# feature_manager = FeatureManager()

In [13]:
# %time F_train = feature_manager.create(session, candidates, 'Sandbox Features')

In [14]:
# from snorkel.utils import get_keys_by_candidate

# for f in get_keys_by_candidate(F_train, F_train.get_candidate(4))[:10]: print f

In [15]:
c = candidates[0]
print c
print c.year
print c.year.parent
print c.year.char_start
print c.year.char_end
print c.year.parent.words
print c.year.parent.char_offsets
print c.year.parent.lemmas
print c.year.parent.pos_tags
print c.year.parent.ner_tags
print c.year.parent.dep_parents
print c.year.parent.dep_labels
print c.year.get_word_start()
print c.year.get_word_end()
# print get_row_ngrams(c.year)
# print get_row_ngrams(c.year, infer=True)
# %time for i in range(1000): _get_aligned_cells((c.year).parent.cell, 'col', infer=True)

Disease(Span("Location", parent=35, chars=[0,7], words=[0,0]))


AttributeError: 'Disease' object has no attribute 'year'

In [ ]:
from snorkel.models import Span, ImplicitSpan, TemporarySpan

print isinstance(c.year, TemporarySpan)

In [ ]:
from snorkel.lf_helpers import get_between_ngrams, get_left_ngrams, get_right_ngrams
from snorkel.lf_helpers import contains_token, contains_regex
from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
from snorkel.lf_helpers import same_document, same_table, same_cell, same_phrase
from snorkel.lf_helpers import _get_aligned_cells, _get_nonempty_cell

In [ ]:
print get_left_ngrams(c.disease)
print get_right_ngrams(c.disease)
print contains_token(c, 'plague')
print contains_regex(c, r'pla')
print same_document(c)
print same_table(c)
print same_cell(c)
print same_phrase(c)
print get_phrase_ngrams(c.disease, n_min=1, n_max=3, case_sensitive=True)
print get_cell_ngrams(c.disease, attrib='pos_tags')
print get_neighbor_cell_ngrams(c.disease, dist=2, directions=True)
print get_row_ngrams(c.disease)
print get_col_ngrams(c.disease)
print get_aligned_ngrams(c.disease)
print get_aligned_ngrams(c.disease, infer=True)

In [ ]:
# from hardware_utils import expand_implicit_text 

# for part in expand_implicit_text(''.join(['BC856/857/858', '/', '859/860'])): print part

In [ ]:
def foo(n):
    phrase = 'repeat me'
    pmul = phrase * n
    pjoi = ''.join([phrase for x in xrange(n)])
    pinc = ''
    for x in xrange(n):
        pinc += phrase
    del pmul, pjoi, pinc

In [ ]:
# %lprun -f foo foo(1000)